## Installing Necessary Packages

In [1]:
!pip install langchain
!pip install langchain-groq
!pip install langchain-huggingface
!pip install cohere
!pip install pinecone
!pip install pypdf
!pip install -U langchain-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 307.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
pip install protoc_gen_openapiv2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.1 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.1 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.1 which is incompatible.
google-cloud-firestore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21

In [3]:
%pip install langchain-community

In [4]:
%pip install langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.10.5
    Uninstalling aiohttp-3.10.5:
      Successfully uninstalled aiohttp-3.10.5


## Importing Liraries

In [6]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRagRetriever, CohereRerank, CohereEmbeddings, ChatCohere
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from google.colab import userdata

cohere_api = userdata.get('COHERE_API_KEY')
import getpass
import os

if not os.getenv("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = userdata.get('COHERE_API_KEY')

## Loading Document

In [7]:
# Create cohere's chat model and embeddings objects
cohere_chat_model = ChatCohere(cohere_api_key=cohere_api)
cohere_embeddings = CohereEmbeddings(cohere_api_key=cohere_api, user_agent="my-app", model="embed-english-v2.0")
# Load text files and split into chunks, you can also use data gathered elsewhere in your application
raw_documents = PyPDFLoader('/content/graph_rag.pdf').load()

In [8]:
len(raw_documents)

9

## Splitting the document into chunks

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200) #
documents = text_splitter.split_documents(raw_documents)
len(documents)

# 1200 characters is a sweet spot where each chunk can encompass enough semantic information (typically 1-2 paragraphs of text)
# a 200-character overlap is selected to avoid redundant processing or unnecessary computational overhead, but it is large enough to capture important connecting information between chunks.


55

In [10]:
## A helper function to show the context from which the answer was generated
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )


## Setting up Pinecone and Creating Index

In [12]:
# loading Pinecone dependencies
import pinecone
import os
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

pinecone_api = userdata.get("PINECONE_API_KEY")
pc = Pinecone(
        api_key=pinecone_api
    )

In [13]:
import time
spec = ServerlessSpec(
    cloud="aws",
    region="us-east-1"
)

index_name = 'docs-rag-chatbot'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=4096,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()


{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Uploading the chunks to Pinecone Vector DB

In [14]:
from langchain_pinecone import PineconeVectorStore
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

index_name = "docs-rag-chatbot"

pinecone = PineconeVectorStore.from_documents(
    documents = documents,
    embedding = cohere_embeddings,
    index_name = index_name,

)

## Setting up Pinecone as Retriever and Quering over it

In [15]:
retriever = pinecone.as_retriever(search_kwargs={"k": 5})
query = "How to do Knowledge Graph Construction?"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

implementation details for our experiments.
2.2 Knowledge Graph Construction
A KG is a structured representation of real-world entities, their
attributes, and their relations, usually stored in a graph database
or a triplet store, i.e., a KG consists of nodes that represent entities
and edges that represent relations, as well as labels and attributes
for both. A graph triplet is a basic unit of information in a KG,
consisting of a subject, a predicate, and an object.
In most methodologies to build a KG from given documents,
three main steps are involved: knowledge extraction, knowledge
1https://www.nseindia.com/products/content/equities/indices/nifty_50.htm
----------------------------------------------------------------------------------------------------
Document 2:

can link these mentions to the same entity node in the graph.
Knowledge Improvement:- This step aims to enhance the quality
and completeness of the KG by removing redundancies and address-
ing gaps in the ex

## Enhancing the RAG Pipeline for more robust retrieval with ReRanking

In [16]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

llm = Cohere(temperature=0)
compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever,
)

compressed_docs = compression_retriever.invoke(
    "How to do Knowledge Graph Construction?"
)

pretty_print_docs(compressed_docs)

<ipython-input-16-0823eeafd2a2>:5: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  llm = Cohere(temperature=0)


Document 1:

implementation details for our experiments.
2.2 Knowledge Graph Construction
A KG is a structured representation of real-world entities, their
attributes, and their relations, usually stored in a graph database
or a triplet store, i.e., a KG consists of nodes that represent entities
and edges that represent relations, as well as labels and attributes
for both. A graph triplet is a basic unit of information in a KG,
consisting of a subject, a predicate, and an object.
In most methodologies to build a KG from given documents,
three main steps are involved: knowledge extraction, knowledge
1https://www.nseindia.com/products/content/equities/indices/nifty_50.htm
----------------------------------------------------------------------------------------------------
Document 2:

HybridRAG: Integrating Knowledge Graphs and Vector Retrieval Augmented Generation for Efficient Information Extraction
Knowledge Graph Construction:- We begin by constructing
a KG from a set of knowledge tri

## Creating the langchain Retrieval Chain with Custom Prompt Template

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are a good assistant that answers questions. Your knowledge is strictly limited to the following piece of context. Use it to answer the question at the end.
    If the answer can't be found in the context, just say you don't know. *DO NOT* try to make up an answer.
    If the question is not related to the context, politely respond that you are tuned to only answer questions that are related to the context.
    **MOST IMPORTANT: If question is not related to the context, just say "I don't know".**

    Context: {context}
    Question: {input}

    """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt_template)



In [19]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(compression_retriever, document_chain)

In [20]:
response = retrieval_chain.invoke({"input":"How to do Knowledge Graph Construction?"})

In [21]:
response

{'input': 'How to do Knowledge Graph Construction?',
 'context': [Document(metadata={'page': 1.0, 'source': '/content/graph_rag.pdf', 'relevance_score': 0.9983451}, page_content='implementation details for our experiments.\n2.2 Knowledge Graph Construction\nA KG is a structured representation of real-world entities, their\nattributes, and their relations, usually stored in a graph database\nor a triplet store, i.e., a KG consists of nodes that represent entities\nand edges that represent relations, as well as labels and attributes\nfor both. A graph triplet is a basic unit of information in a KG,\nconsisting of a subject, a predicate, and an object.\nIn most methodologies to build a KG from given documents,\nthree main steps are involved: knowledge extraction, knowledge\n1https://www.nseindia.com/products/content/equities/indices/nifty_50.htm'),
  Document(metadata={'page': 6.0, 'source': '/content/graph_rag.pdf', 'relevance_score': 0.9965436}, page_content='HybridRAG: Integrating Know

## Checking the Retireval Pipeline on different Queries

In [22]:
print(response["answer"])

The knowledge graph construction process involves three main steps: knowledge extraction, knowledge improvement, and knowledge adaptation. 

During the knowledge extraction step, structured information is extracted from unstructured or semi-structured data like text, databases, and existing ontologies. This step involves entity recognition, relationship extraction, and co-reference resolution tasks to identify entities and their relationships from textual sources. These techniques allow for the identification of entities and the relationships between them, which can be used to create nodes and edges in a knowledge graph. 

The knowledge improvement step aims to enhance the quality and completeness of the knowledge graph by removing redundancies and addressing any inconsistencies or coherence issues within the graph. 

Lastly, knowledge adaptation treats the knowledge graph as a static graph, where the information is not adapted or updated unless specifically prompted to do so. 

Due to

In [23]:
response2 = retrieval_chain.invoke({"input":"Who is the prime minister of India?"})
print(response2["answer"])

The prime minister of India is Narendra Modi. I don't know when he took office or the current political landscape of India.  I don't know. 


In [24]:
response3 = retrieval_chain.invoke({"input":"What is Machine Learning?"})
print(response3["answer"])

I don't know. The provided context does not include any information regarding the definition of Machine Learning. If you like, I can provide some information on the topic if you have specific questions relating to it. 


In [25]:
response4 = retrieval_chain.invoke({"input":"What is Retrieval Augmented Generation?"})
print(response4["answer"])

Retrieval Augmented Generation (RAG) is a method to enhance the performance of large language models (LLMs) by incorporating relevant retrieval techniques. RAG methods retrieve pertinent documents or text to provide additional context during the generation process, ensuring the LLMs generate relevant and coherent responses that align with the original input query. Different types of RAG techniques include VectorRAG, Realm, and HybridRAG, each with its own strengths and limitations. 
